<a href="https://colab.research.google.com/github/kspc-ai/learn-collab/blob/main/Learn_Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Gemini API — Beginner Google Colab Project

**Goal:** A step-by-step Colab notebook that shows how to use **Gemini API keys** to call Google’s Gemini models (text generation) safely and simply. This is written for beginners and includes detailed explanations and commented code you can paste directly into a new Colab cell.

---

## Quick overview

What you'll learn:

1. Where to get a Gemini API key (Google AI Studio).
2. How to install the official Python client in Colab.
3. Two safe ways to provide the API key in Colab (hidden input / environment variable).
4. Make a simple text-generation call using the `google-genai` client.
5. Make the equivalent REST call with `requests` using the `x-goog-api-key` header.
6. Security and troubleshooting tips.

---

## Prerequisites

* A Google account with access to **Google AI Studio** (where you can create a Gemini API key).
* Basic comfort running cells in Google Colab.
* This notebook is intentionally simple and uses only `pip install` inside Colab (no Cloud SDK required).

---

## 0) Open a new Google Colab

1. Go to [https://colab.research.google.com](https://colab.research.google.com)
2. Create a new Python 3 notebook.

Copy each code block below into its own Colab cell and run it in order.

---

## 1) Get a Gemini API key (short summary)

1. Open **Google AI Studio** (the Gemini pages) and sign in with your Google account.
2. Find **API Keys** (or "Get API key") in the left sidebar and create a key for a project.
3. Copy the key; store it somewhere safe (treat it like a password).

> **Do not paste your key into public GitHub repositories or share it.**

---

## 2) Install the official Python client in Colab

In [15]:
# Colab cell: install the official Google GenAI package
!pip install -q -U google-genai

# (Optional) we'll import standard libs in the next cell

In [16]:
# Colab cell: enter API key without echoing it in the notebook
from getpass import getpass
import os

# Prompt (hidden): paste your Gemini API key here when asked
api_key = getpass("Enter your Gemini API key (input will be hidden): ")

# Put the key into environment so client libraries that auto-discover env vars can use it
os.environ['GEMINI_API_KEY'] = api_key

# You can also set GOOGLE_API_KEY if you prefer; GEMINI_API_KEY is recommended.
# NOTE: This stores key only in the current Colab session memory; it is not saved to the notebook file.

Enter your Gemini API key (input will be hidden): ··········


In [17]:
# Later we'll show using the explicit API key when constructing the client.
# This avoids relying on environment variable auto-discovery.

from getpass import getpass
api_key = getpass("Enter your Gemini API key (input will be hidden): ")

# We'll use this `api_key` directly when creating the client.

Enter your Gemini API key (input will be hidden): ··········


In [18]:
# Colab cell: minimal text generation using the python client
from google import genai
import os

# Option 1: auto-discover API key from environment:
# os.environ['GEMINI_API_KEY'] should already be set by the earlier cell if you used getpass.
client = genai.Client()  # client will read GEMINI_API_KEY or GOOGLE_API_KEY if available

# Option 2 (explicit):
# client = genai.Client(api_key=api_key)

# Choose a model (examples include 'gemini-2.5-flash', 'gemini-1.5-mini'; pick what's available to you)
MODEL = "gemini-2.5-flash"

try:
    response = client.models.generate_content(
        model=MODEL,
        contents="Explain 'overfitting' in machine learning in 2-3 sentences."
    )

    # The 'response' object offers a convenient .text or .content field depending on the SDK version.
    print("--- Generated text ---")
    print(response.text)

except Exception as e:
    print("An error occurred while calling the Gemini API:", e)
    print("Common causes: invalid API key, project not imported in AI Studio, or network issues.")

--- Generated text ---
Overfitting occurs when a machine learning model learns the training data too well, including its noise and specific details, rather than the underlying general patterns. This leads to excellent performance on the training set but poor generalization and low accuracy when encountering new, unseen data.


In [19]:
# Colab cell: make a direct REST call to the Gemini endpoint with the API key in the header
import requests
import os

api_key = os.environ.get('GEMINI_API_KEY')
endpoint = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

payload = {
    "contents": [
        {
            "parts": [{"text": "Explain the benefits of unit testing in 3 bullet points."}]
        }
    ]
}

headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": api_key  # REST header required for API key authentication
}

resp = requests.post(endpoint, headers=headers, json=payload)

if resp.status_code == 200:
    print("REST response JSON:\n", resp.json())
else:
    print("Request failed (status):", resp.status_code)
    print(resp.text)

REST response JSON:
 {'candidates': [{'content': {'parts': [{'text': 'Here are three key benefits of unit testing:\n\n*   **Catches Bugs Early:** Unit tests identify defects in small, isolated pieces of code at the earliest possible stage of development, making them significantly cheaper and easier to fix than finding them later in integration or production.\n*   **Improves Code Quality and Design:** Writing unit tests encourages developers to write more modular, testable, and maintainable code. It acts as a form of living documentation, clearly defining what each unit of code is supposed to do.\n*   **Enables Safe Refactoring and Future Changes:** A comprehensive suite of unit tests provides a safety net. Developers can confidently refactor existing code, add new features, or change implementations, knowing that if they inadvertently break existing functionality, the tests will immediately alert them.'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0}], 'usageMetadata': {'promp

In [20]:
# Colab cell: small helper to wrap repeated calls and print results cleanly
from google import genai
import os

client = genai.Client()  # auto-discover env var or use explicit key

def generate_short_text(prompt, model="gemini-2.5-flash"):
    """Return the model's text for a short prompt (wraps error handling)."""
    try:
        resp = client.models.generate_content(model=model, contents=prompt)
        return resp.text
    except Exception as e:
        return f"Error: {e}"

# Example usage
print(generate_short_text("What's a simple explanation of cosine similarity?"))

Imagine you have two documents (like two articles, two customer reviews, or even just two sentences). You want to know how similar they are in terms of *what they are talking about*.

Cosine similarity is a way to measure this similarity, and here's the simple breakdown:

1.  **Turn things into "vectors":** First, you take your documents and convert them into numerical representations called "vectors." Think of a vector as an arrow in a multi-dimensional space. Each dimension might represent a word, and the "length" along that dimension tells you how often that word appears in the document.

    *   **Example:**
        *   Document A: "The quick brown fox." -> Vector A: [quick:1, brown:1, fox:1, the:1]
        *   Document B: "The lazy dog." -> Vector B: [lazy:1, dog:1, the:1]
        *   Document C: "The quick fox jumps over the quick brown fox." -> Vector C: [quick:2, brown:1, fox:2, jumps:1, over:1, the:2]

2.  **Focus on the "direction," not the "length":**
    *   Cosine similari